## 영화 페이지 댓글로 유사도 분석
* 네이버 영화에서 <다만 악에서 구하소서> 댓글을 scrapping
* 댓글에서 단어만 추출
* 유사도 측정
* 특정 단어의 유사도 측정
* https://movie.naver.com/movie/bi/mi/point.nhn?code=189069#tab

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

In [2]:
import re

# 클리닝 함수 (file에 특수문자가 있을 경우 )

def clean_text(text):
    text = text.replace("\n", "")
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
    return cleaned_text

In [3]:
#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [4]:
main_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=189069&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3'
res = requests.get(main_url)
html = res.text

soup = BeautifulSoup(html, 'html.parser')
ptag_list = []
for p_tag in soup.select(".score_result .score_reple"):
    result = p_tag.text
    result = result.replace('\t','')
    result = result.replace('\r','')
    result = result.replace('\n','')
    result = clean_text(result.replace('관람객', '').split('2020.')[0]).strip()
    ptag_list.append(result)

ptag_list

['박정민은 진심 올해의 히든카드였다 황정민이정재 사이에서 존재감 제대로 각인됨 20121201',
 '왜 내가 다 아픈기분이지 ㅋㅋㅋ 타격감이 그대로 느껴짐 ㅋㅋ 박민수',
 '박정민쵝오입니다믿고보는 기대가되는 배우 고승혁',
 '복도에서 마주친 그순간이 제일 재밌었음',
 '박정민 대박ㅋㅋㅋ 재밌고 시간 가는줄 모르고 보게 됨 누룽지탕',
 '반도따위랑 비교하지마라그냥올해본거중에 원탑 승현',
 '개봉하자마자 보고 왔는데 재밌습니다 적어도 최근 개봉한 영화들중에선 이게 제일 괜찮아요 아키라',
 '반도강철비다만악에서 구하소서 장원준1',
 '황정민 이정재 박정민을 데리고 왜 이런 개밥바라기 같은 영화를 만든거죠 말해봐요 왜그랬어요',
 '최근 나온 한국 영화들 중 최고입니다 우드0']

In [5]:
comment_result = []

for i in ptag_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

comment_result

['박정민은 진심 올해의 히든카드였다 황정민이정재 사이에서 존재감 제대로 각인됨 20121201',
 '왜 내가 다 아픈기분이지  타격감이 그대로 느껴짐  박민수',
 '박정민쵝오입니다믿고보는 기대가되는 배우 고승혁',
 '복도에서 마주친 그순간이 제일 재밌었음',
 '박정민 대박 재밌고 시간 가는줄 모르고 보게 됨 누룽지탕',
 '반도따위랑 비교하지마라그냥올해본거중에 원탑 승현',
 '개봉하자마자 보고 왔는데 재밌습니다 적어도 최근 개봉한 영화들중에선 이게 제일 괜찮아요 아키라',
 '반도강철비다만악에서 구하소서 장원준1',
 '황정민 이정재 박정민을 데리고 왜 이런 개밥바라기 같은 영화를 만든거죠 말해봐요 왜그랬어요',
 '최근 나온 한국 영화들 중 최고입니다 우드0']

In [6]:
!pip install konlpy

In [7]:
!pip show konlpy
# !pip install --upgrade pip

Name: konlpy
Version: 0.5.2
Summary: Python package for Korean natural language processing.
Home-page: http://konlpy.org
Author: Team KoNLPy
Author-email: konlpy@googlegroups.com
License: GPL v3
Location: c:\python37\lib\site-packages
Requires: JPype1, colorama, numpy, lxml, beautifulsoup4, tweepy
Required-by: 


In [8]:
from konlpy.tag import Kkma

def get_noun(comment_txt):
    twitter = Twitter()
    noun = []
    
    if len(comment_txt)>0:
        tw = twitter.pos(comment_txt)
        for i,j in tw:
            if j == 'Noun':
                noun.append(i)
    return noun

kkma = Kkma()
# kkma.nouns('연기는 손색이 없으나 스토리가 조금 빈약하지 않나 생각합니다 뽈티')
noun_list = []
for x in comment_result:
    noun_list += kkma.nouns(x)

noun_list = [a for a in noun_list if len(a) > 1]    
noun_list

ModuleNotFoundError: No module named 'konlpy'

In [9]:
!pip install WordCloud

In [10]:
!pip show WordCloud

Name: wordcloud
Version: 1.7.0
Summary: A little word cloud generator
Home-page: https://github.com/amueller/word_cloud
Author: Andreas Mueller
Author-email: t3kcit+wordcloud@gmail.com
License: MIT
Location: c:\python37\lib\site-packages
Requires: numpy, matplotlib, pillow
Required-by: 


In [15]:
from collections import Counter
from WordCloud import wordcloud
# import pytagcloud, pygame
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

counts = Counter(noun_list)
tags = counts.most_common(30)

#bar chart
test = pd.DataFrame({'word':[],
                    'count':[]})
for i in range(len(tags)):
    word = tags[i][0]
    count = tags[i][1]
    
    insert_data = pd.DataFrame({'word':[word],
                                'count':[count]})
    test = test.append(insert_data)

test.index = range(len(test))

index = np.arange(len(test))
plt.bar(index,test['count'].tolist() )
plt.xlabel('word', fontsize=5)
plt.ylabel('count', fontsize=5)
plt.xticks(index, test['word'].tolist(), fontsize=5, rotation=30)
plt.title('단어 빈도수 시각화')
plt.show()

#wordcloud
# taglist = pytagcloud.make_tags(tags, maxsize=50)
# pytagcloud.create_tag_image(taglist, 'wordcloud.jpg', size=(600, 600), fontname='Korean', rectangular=False)

#wordcloud
wc = WordCloud(font_path='C:\\Windows\\Fonts\\NanumGothic.ttf',background_color='white', width=800, height=600)

print(dict(tags))
cloud = wc.generate_from_frequencies(dict(tags))
plt.figure(figsize=(10, 8))
plt.axis('off')
plt.imshow(cloud)
plt.show()

ModuleNotFoundError: No module named 'WordCloud'

In [16]:
from gensim.models import Word2Vec

vec = []
for x in comment_result:
    tmp = [n for n in kkma.nouns(x) if len(n)>1]
    vec.append(tmp)

model = Word2Vec(vec, min_count=3,window=3,iter=20, size=100, sg=1) #skip-gram
model.most_similar('연기')
# model.most_similar('스토리')
# model.most_similar('액션')

ModuleNotFoundError: No module named 'gensim'

In [ ]:
!pip install gensim